In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate
import torch
from sklearn.model_selection import train_test_split
from torch import optim
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
train_df = pd.read_csv("./C01_train_encoded.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182080 entries, 0 to 182079
Columns: 1832 entries, id to description_enc_486
dtypes: float64(5), int64(1826), object(1)
memory usage: 2.5+ GB


In [3]:
train_df.head(5)

,id,project_is_approved,project_grade_category_enc_0,project_grade_category_enc_1,project_grade_category_enc_2,project_grade_category_enc_3,school_state_enc_0,school_state_enc_1,school_state_enc_2,school_state_enc_3,...,description_enc_477,description_enc_478,description_enc_479,description_enc_480,description_enc_481,description_enc_482,description_enc_483,description_enc_484,description_enc_485,description_enc_486
0,p036502,1.0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,p039565,0.0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,p233823,1.0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,p185307,0.0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,p013780,1.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# 全体の30%は検証用
train_data = train_df.values
X = train_data[:, 2:].astype(np.float)  # 正解データ列以外を抽出
Y = train_data[:, 1].astype(np.long)  # 正解データ列のみ抽出

X_train, X_test, Y_train, Y_test = train_test_split(
X, Y, test_size=0.3)

X_train = torch.tensor(X_train, dtype=torch.float)
Y_train = torch.tensor(Y_train, dtype=torch.long) 
X_test = torch.tensor(X_test, dtype=torch.float)
Y_test = torch.tensor(Y_test, dtype=torch.long) 

In [5]:
net = nn.Sequential(
    nn.Linear(X.shape[1], 1024),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 2)
)

In [6]:
#X_train = X_train.to("cuda:0")
#Y_train = Y_train.to("cuda:0")
#X_test = X_test.to("cuda:0")
#Y_test = Y_test.to("cuda:0")
#net.to("cuda:0")

In [7]:
# ソフトマックスクロスエントロピー
loss_fn = nn.CrossEntropyLoss()

# Adam
optimizer = optim.Adam(net.parameters())

# 損失関数のログ
train_losses = []
test_losses = []

# 訓練用データでDataLoaderを作成
ds = TensorDataset(X_train, Y_train)
loader = DataLoader(ds, batch_size=256, shuffle=True)

In [8]:
for epoch in range(10):
    running_loss = 0.0
    for i, (xx, yy) in enumerate(loader):
        y_pred = net(xx)
        loss = loss_fn(y_pred, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / i)
    y_pred = net(X_test)
    test_loss = loss_fn(y_pred, Y_test)
    test_losses.append(test_loss.item())

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(train_losses)

In [ ]:
plt.plot(test_losses)